In [1]:
from data_utils import get_karpathy_split
from data_loader_captions import get_caption_loader
from build_vocab import Vocabulary

import nltk
from string import punctuation
from tqdm.autonotebook import tqdm

import torchvision.transforms as transforms

[nltk_data] Downloading package punkt to /home/vu48pok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
splits_path = '/home/vu48pok/.data/compling/data/corpora/external/MSCOCO/COCO/splits/karpathy/caption_datasets/'
caps_path = '/home/vu48pok/.data/compling/data/corpora/external/MSCOCO/COCO/'
image_dir = '/home/vu48pok/.data/compling/data/corpora/external/MSCOCO/COCO/'
crop_size=224

In [3]:
caps_df = get_karpathy_split(splits_path=splits_path, caps_path=caps_path)

In [4]:
vocab = Vocabulary()

vocab.add_word('<pad>')
vocab.add_word('<start>')
vocab.add_word('<end>')
vocab.add_word(vocab.unknown_token)

train_caps = caps_df.loc[caps_df.split == 'train'].caption.to_list()
for c in tqdm(train_caps):
    c = c.casefold()
    c = c.translate(str.maketrans('', '', punctuation))
    for t in nltk.word_tokenize(c):
        vocab.add_word(t)

In [5]:
transform = transforms.Compose([
        transforms.Resize((crop_size, crop_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))
])

In [6]:
loader = get_caption_loader(
    decoding_level='word', 
    split='train',
    data_df=caps_df, 
    image_dir=image_dir, 
    vocab=vocab,
    transform=transform, 
    batch_size=32, 
    shuffle=False,
    num_workers=2, 
    drop_last=False   
)

In [15]:
caps_df

,image_id,caption,coco_split,split
id,,,,
48,318556,A very clean and well decorated empty bathroom,train,train
67,116100,A panoramic view of a kitchen and all of its a...,train,train
126,318556,A blue and white bathroom with butterfly theme...,train,train
148,116100,A panoramic photo of a kitchen and dining room,train,train
173,379340,A graffiti-ed stop sign across the street from...,train,train
...,...,...,...,...
828740,401092,A plate of food and a beverage are on a table.,val,restval
828842,401092,This is an open faced sandwich with several co...,val,restval
828909,555904,People eating in a restaurant near wine bottles.,val,restval


In [11]:
for i, data in enumerate(loader):
    d = data
    if i > 0:
        break

In [12]:
images, targets, lengths = d

In [17]:
from torch.nn.utils.rnn import pack_padded_sequence

In [26]:
targets[:, 1:]

tensor([[  4,  15,  21,   4,  63,  64, 105,  86,  21, 213, 214,   4, 215,   4,
          57,   7,   4, 216,   2],
        [ 26,  14,   4, 171, 139,  48, 242, 243, 129,  80, 244, 245, 163, 246,
          33, 247, 248, 249,   2],
        [  4, 175, 176,   7,   4, 171, 177, 178,   4, 179,  40,   4, 180, 129,
           2,   0,   0,   0,   0],
        [  4, 171, 199, 111,   4,  15, 200,  78,   4,  28, 201, 101, 202,   2,
           0,   0,   0,   0,   0],
        [  4,  61, 115, 223,  48, 224, 225, 209, 226,  40,  33,  11,  64,   2,
           0,   0,   0,   0,   0],
        [  4,  98,  40,   4, 129, 130, 139,  78, 121,  14,   4,  37,   2,   0,
           0,   0,   0,   0,   0],
        [253, 130, 254, 255, 109,  78,  33,  64,  78,  48,  10,  15,   2,   0,
           0,   0,   0,   0,   0],
        [ 33, 260,  14,   4,  15, 154, 261,  21, 262, 263,   7, 264,   2,   0,
           0,   0,   0,   0,   0],
        [  4,  91, 130, 265, 266, 225, 209, 226,  40,   4, 267, 132,   2,   0,
         

In [27]:
for t in targets[:,1:]:
    print(' '.join([vocab.idx2word[i.item()] for i in t]))

a kitchen with a tile floor has cabinets with no doors a dishwasher a sink and a refrigerator <end>
photo of a man riding an old styled bicycle near what appears to be the golden gate bridge <end>
a purple bus and a man dressed as a nun on a tall bicycle <end> <pad> <pad> <pad> <pad>
a man working at a kitchen counter in a room illuminated by sunlight <end> <pad> <pad> <pad> <pad> <pad>
a black cat making an angry face while sitting on the bathroom floor <end> <pad> <pad> <pad> <pad> <pad>
a person on a bicycle is riding in front of a car <end> <pad> <pad> <pad> <pad> <pad> <pad>
there is some pet food in the floor in an empty kitchen <end> <pad> <pad> <pad> <pad> <pad> <pad>
the top of a kitchen cabinet covered with brass pots and pans <end> <pad> <pad> <pad> <pad> <pad> <pad>
a woman is shaving her face while sitting on a wooden bench <end> <pad> <pad> <pad> <pad> <pad> <pad>
a cat peeking out a car window that is rolled down <end> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
a closeup 